#### Creating a custom Rail Yard gym environment

We first create a custome rail yard environment or "game". We are using the Open AI Gym library which is intended to help test out different AI agents on various game like environments. We created this environment by subclassing the Gym evironment class and overiding key methods such as __init__(), step(), and reset().

In [1]:
import gym
import RailYardGymEnv
env_name = 'RailYardGymEnv-v0'
rail_yard_env = gym.make(env_name)

The environment is a text based game. The objective is to load the cars as per the schedule in as few steps as possible.  Each step you take you lose -10 points.  If you load all the required cars you get 1000 points.  Let's reset the environment and see the loading schedule:

In [2]:
rail_yard_env.reset()
print("Loading Schedule:")
print(str(rail_yard_env.ry.loading_schedule) + "\n")

Loading Schedule:
Set: 1
Car: m2 Product:M
Car: d3 Product:D
Set: 2
Car: d4 Product:D




As you can see the schedule is broken up into sets.  Each set lists the car number and product to load (e.g. Car: m2 Product:M).  

Now let's create a main game loop for this environment and play the game!  Note that the environment expects an action at each step of the game that indicates how many cars to move from a source to destination track. The user can also "wait" or "skip the turn" by entering a blank for any of the fields. Finally, if there are insufficient cars, spots or the tracks do not connect the moves are considered invalid and result in a loss of -10 points.

The environment returns an observation,m 

In [3]:
def run_railyard_game(rail_yard_env):
    print(rail_yard_env.render())
    done = False
    while not done:
        #get input from user
        from_track = input("From track: ")
        to_track = input("To track: ")
        num_cars = input("Number of cars: ")
        if from_track == "" or to_track  == "" or num_cars == "" :
            action = 0 #DO_NOTHING_ACTION
        else:    
            action = rail_yard_env.encode_action(int(from_track), int(to_track), int(num_cars))

        observation, reward, done, info = rail_yard_env.step(action)
        print(rail_yard_env.render())
        
run_railyard_game(rail_yard_env)

Period: 0
Track 1: []
Track 2: ['m1', 'm2', 'd3', 'd4']
Track 3: []
Track 4: []
Rack 5: []
Rack 6: []
Track 7: []

From track: 2
To track: 1
Number of cars: 4
Period: 1
Track 1: ['d4', 'd3', 'm2', 'm1']
Track 2: []
Track 3: []
Track 4: []
Rack 5: []
Rack 6: []
Track 7: []

From track: 1
To track: 5
Number of cars: 2
Period: 2
Track 1: ['d4', 'd3']
Track 2: []
Track 3: []
Track 4: []
Rack 5: ['m1', 'm2']
Rack 6: []
Track 7: []

From track: 1
To track: 6
Number of cars: 2
Period: 3
Track 1: []
Track 2: []
Track 3: []
Track 4: []
Rack 5: ['m1', 'm2']
Rack 6: ['d3', 'd4']
Track 7: []

From track: 
To track: 
Number of cars: 
Period: 4
Track 1: []
Track 2: []
Track 3: []
Track 4: []
Rack 5: ['m1', 'm2']
Rack 6: ['d3', 'd4']
Track 7: []

From track: 
To track: 
Number of cars: 
Period: 5
Track 1: []
Track 2: []
Track 3: []
Track 4: []
Rack 5: ['M1', 'M2']
Rack 6: ['d3', 'd4']
Track 7: []

From track: 5
To track: 1
Number of cars: 2
Period: 6
Track 1: ['M2', 'M1']
Track 2: []
Track 3: []
Trac

In [4]:
rail_yard_env.reset()

array([[2, 0, 0, 0, 0],
       [2, 1, 0, 1, 0],
       [2, 2, 0, 1, 1],
       [2, 3, 0, 2, 1]])

Each row in our observation corresponds to a rail car and is of the form: (track number, position on track, loaded or empty, set number and product if on loading schedule).  Thus the second row in the matrix  [2, 1, 0, 1, 0] means that rail car 2 is on track number 2, in spot 1, it is empty, it is scheduled for loading on set 1, and should be loaded with product 0 (M).

Let's take an action in the environment and see how the observation changes:

In [5]:
rail_yard_env.step(rail_yard_env.encode_action(2,1,2))

(array([[2, 0, 0, 0, 0],
        [2, 1, 0, 1, 0],
        [1, 1, 0, 1, 1],
        [1, 0, 0, 2, 1]]),
 -10,
 False,
 None)

We "popped" 2 cars from track 2 and "pushed" them onto 1.  Note we used the encode_action() method in our custom gym environment which assigns a unique integer to every possible action in our discrete action space.

Let's start with the smallest problem of just 3 tracks, 1 car and 1 product to load.

In [6]:
env_name = 'MinScenarioRailYardGymEnv-v0'
rail_yard_env = gym.make(env_name)
rail_yard_env.reset()
print("Loading Schedule:")
print(str(rail_yard_env.ry.loading_schedule) + "\n")
run_railyard_game(rail_yard_env)

Loading Schedule:
Set: 1
Car: m1 Product:M


Period: 0
Track 1: ['m1']
Rack 2: []
Track 3: []

From track: 1
To track: 2
Number of cars: 1
Period: 1
Track 1: []
Rack 2: ['m1']
Track 3: []

From track: 
To track: 
Number of cars: 
Period: 2
Track 1: []
Rack 2: ['m1']
Track 3: []

From track: 
To track: 
Number of cars: 
Period: 3
Track 1: []
Rack 2: ['m1']
Track 3: []

From track: 
To track: 
Number of cars: 
Period: 4
Track 1: []
Rack 2: ['M1']
Track 3: []

From track: 2
To track: 1
Number of cars: 1
Period: 5
Track 1: ['M1']
Rack 2: []
Track 3: []

From track: 1
To track: 3
Number of cars: 1
Period: 6
Track 1: []
Rack 2: []
Track 3: ['M1']



#### Wrapping the environment for use by TF Agents

Here we are going to start using the TensorFlow Agents library. We load and wrap our gym environment in a Tensorflow wrapper.  This allows us to exchange obserations, actions, and rewards between the agent and the environment as tensors and be callable from within a high performance graph code. Note we typically create one environment for training and one for evaluation.

In [7]:
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment

train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)
train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

The reset method returns an observation in a TimeStep object.

In [8]:
train_env.reset()
eval_env.reset()

TimeStep(step_type=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>, reward=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>, discount=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>, observation=<tf.Tensor: shape=(1, 1, 5), dtype=int32, numpy=array([[[1, 0, 0, 1, 0]]])>)

We can also see the specifications of our observation and action space.  These are used to configure the agent network input and out layers so that we can vary the environment implementation without impacting the agents.

In [9]:
train_env.observation_spec()

BoundedTensorSpec(shape=(1, 5), dtype=tf.int32, name='observation', minimum=array(0), maximum=array(4))

In [10]:
train_env.action_spec()

BoundedTensorSpec(shape=(), dtype=tf.int64, name='action', minimum=array(0, dtype=int64), maximum=array(9, dtype=int64))

#### Creating a DQN agent

A TF-Agents training program is split into an environment exploration part (a driver that explores the environment using a collect policy and sends trajectories (observations, actions, rewards) to one or more observers who write to a replay buffer) and a learning part (an agent who pulls batches off the replay buffer and trains networks to be leveraged by the collect policy and the final agent policy).

We first create a Q network with a fully connected layer of 32 neurons

In [11]:
from tf_agents.networks import q_network
fc_layer_params = [32]
q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params)

We then use the network to create a DQN agent which uses 2 Q networks (one for exploration and one for setting targets in loss calculations) and an epsilon greedy policy for exploration.  Note we also provide a gradient based optimizer for performnig updates to the network weights to minimize the loss, the loss function (Huber loss which is just the squared different of the target vs current estimate of the value state/action function), and the 

In [12]:
import tensorflow as tf
from tensorflow import keras
from tf_agents.agents.dqn.dqn_agent import DqnAgent

num_iterations = 200000
learning_rate = 2.5e-3 #how fast to update the Q value function
initial_e = 0.5 #initial epsilon
final_e = 0.01 #final epsilon
target_update_period = 100 #how many training iterations to play between every update to the target Q value function

train_step = tf.Variable(0,dtype=tf.int64) #keeps track of number of training iterations

optimizer = tf.compat.v1.train.RMSPropOptimizer(learning_rate=learning_rate, decay=0.95, momentum=0.0,
                                     epsilon=0.00001, centered=True)

epsilon_fn = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=initial_e, # initial ε
    decay_steps=num_iterations,
    end_learning_rate=final_e) # final ε

agent = DqnAgent(train_env.time_step_spec(),
                 train_env.action_spec(),
                 q_network=q_net,
                 optimizer=optimizer,
                 #observation_and_action_constraint_splitter=observation_and_action_constraint_splitter,
                 target_update_period=100,
                 td_errors_loss_fn=keras.losses.Huber(reduction="none"),
                 gamma=0.99, # discount factor
                 train_step_counter=train_step,
                 epsilon_greedy=lambda: epsilon_fn(train_step))
agent.initialize()

Let's create a replay buffer to store the trajectories (experience) collected by the driver and passed to the observer.

In [13]:
from tf_agents.replay_buffers import tf_uniform_replay_buffer
replay_buffer_max_length = 10000

# A buffer to stores the experience from interacting with the environment
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)

Let's create the observer that writes the experience to the replay buffer

In [14]:
#An observer to write the trajectories to the buffer
replay_buffer_observer = replay_buffer.add_batch

Let's create the metrics to be written to the screen and tensorboard

In [15]:
from tf_agents.metrics import tf_metrics
import logging
#Training metrics
train_metrics = [
    tf_metrics.NumberOfEpisodes(),
    tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(),
    tf_metrics.AverageEpisodeLengthMetric()
]
logging.getLogger().setLevel(logging.INFO)

Now let's create the driver to explore the environment.  The driver will use the collect policy and pass the experience trajectories from num_steps to the replay_buffer_observer.add_batch and all the train_metrics functions.

In [16]:
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
collect_steps_per_iteration = 1 #how many steps to play in each training iteration
#Create the driver
collect_driver = DynamicStepDriver(
    train_env,
    agent.collect_policy,
    observers=[replay_buffer_observer] + train_metrics,
    num_steps=collect_steps_per_iteration) # collect # steps for each training iteration

Finally create a data set from the buffer that the agent will use to pull records for training

In [17]:
batch_size = 32
# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, 
    sample_batch_size=batch_size, 
    num_steps=2).prefetch(3)

Create the main training loop and convert it to a graph function for high performance execution

In [18]:
import datetime
from tf_agents.utils.common import function
from tf_agents.eval.metric_utils import log_metrics

def train_agent(n_iterations):
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    train_log_dir = 'logs/dqn_agent/' + current_time + '/train'
    writer = tf.summary.create_file_writer(train_log_dir)
    time_step = None
    policy_state = agent.collect_policy.get_initial_state(train_env.batch_size)
    iterator = iter(dataset)
    with writer.as_default():
        for iteration in range(n_iterations):
            time_step, policy_state = collect_driver.run(time_step, policy_state)
            trajectories, buffer_info = next(iterator)
            train_loss = agent.train(trajectories)
            #log metrics
            print("\r{} loss:{:.5f}".format(iteration, train_loss.loss.numpy()), end="")
            if iteration % 1000 == 0:
                log_metrics(train_metrics)
                tf.summary.scalar("number_of_episodes", train_metrics[0].result(), iteration)
                tf.summary.scalar("environment_steps", train_metrics[1].result(), iteration)
                tf.summary.scalar("average_return", train_metrics[2].result(), iteration)
                tf.summary.scalar("average_episode_length", train_metrics[3].result(), iteration)
                writer.flush()
                
collect_driver.run = function(collect_driver.run)
agent.train = function(agent.train)

We are now ready to start the training.  The first thing we will do is prefill the buffer with actions following the optimal policy (this is a very simple environment and we know it!). We also create a utility observer to track progress on the initial run.

In [19]:
class ShowProgress:
    def __init__(self, total):
        self.counter = 0
        self.total = total
    def __call__(self, trajectory):
        if not trajectory.is_boundary():
            self.counter += 1
        if self.counter % 100 == 0:
            print("\r{}/{}".format(self.counter, self.total), end="")
            
from min_scenario_policy import MinYardScenarioPolicy
pretrain_steps = 10000 #number of steps to initialize the buffer with a pre trained policy
initial_collect_policy = MinYardScenarioPolicy(train_env.time_step_spec(),train_env.action_spec())

init_driver = DynamicStepDriver(
    train_env,
    initial_collect_policy,
    observers=[replay_buffer.add_batch,ShowProgress(pretrain_steps)],
    num_steps=pretrain_steps)
final_time_step, final_policy_state = init_driver.run()

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
10000/10000

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/dqn_agent

Finally let's run the training loop...

In [20]:
train_agent(num_iterations)

INFO:absl: 
		 NumberOfEpisodes = 0
		 EnvironmentSteps = 1
		 AverageReturn = 0.0
		 AverageEpisodeLength = 0.0


996 loss:35.950918

INFO:absl: 
		 NumberOfEpisodes = 12
		 EnvironmentSteps = 1001
		 AverageReturn = -268.0
		 AverageEpisodeLength = 77.30000305175781


1928 loss:617.93292

INFO:absl: 
		 NumberOfEpisodes = 23
		 EnvironmentSteps = 2001
		 AverageReturn = -707.0
		 AverageEpisodeLength = 90.9000015258789


...

197977 loss:59.401369

INFO:absl: 
		 NumberOfEpisodes = 5842
		 EnvironmentSteps = 198001
		 AverageReturn = 841.0
		 AverageEpisodeLength = 16.899999618530273


198990 loss:33.583356

INFO:absl: 
		 NumberOfEpisodes = 5879
		 EnvironmentSteps = 199001
		 AverageReturn = 831.0
		 AverageEpisodeLength = 17.899999618530273


199999 loss:0.1344757

Let's evaluate the agent to see if it can solve the rail yard on its own

In [ ]:
#how many episodes to play to evaluate the agent 
num_eval_episodes = 5 

def create_policy_eval_text_log(policy, filename, num_episodes=5):
    logfile = open(filename + ".txt","w")
    for _ in range(num_episodes):
        time_step = eval_env.reset()
        logfile.write(eval_py_env.render())
        while not time_step.is_last():
            action_step = policy.action(time_step)
            time_step = eval_env.step(action_step.action)
            logfile.write("=============================\n")
            logfile.write("Action: " + str(eval_py_env.decode_action(action_step.action.numpy()[0])) + "\n")
            logfile.write(eval_py_env.render())

create_policy_eval_text_log(agent.policy, "trained-agent",num_eval_episodes)

In [ ]:
# %load trained-agent.txt
Period: 0
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 1
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 2
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 3
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 4
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 5
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 6
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 7
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 8
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 9
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 10
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 11
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 12
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 13
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 14
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 15
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 16
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 17
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 18
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 19
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 20
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 21
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 22
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 23
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 24
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 25
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 26
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 27
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 28
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 29
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 30
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 31
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 32
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 33
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 34
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 35
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 36
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 37
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 38
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 39
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 40
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 41
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 42
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 43
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 44
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 45
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 46
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 47
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 48
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 49
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 50
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 51
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 52
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 53
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 54
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 55
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 56
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 57
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 58
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 59
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 60
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 61
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 62
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 63
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 64
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 65
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 66
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 67
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 68
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 69
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 70
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 71
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 72
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 73
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 74
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 75
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 76
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 77
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 78
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 79
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 80
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 81
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 82
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 83
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 84
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 85
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 86
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 87
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 88
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 89
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 90
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 91
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 92
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 93
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 94
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 95
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 96
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 97
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 98
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 99
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 100
Track 1: ['m1']
Rack 2: []
Track 3: []
Period: 0
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 1
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 2
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 3
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 4
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 5
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 6
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 7
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 8
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 9
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 10
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 11
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 12
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 13
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 14
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 15
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 16
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 17
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 18
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 19
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 20
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 21
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 22
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 23
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 24
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 25
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 26
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 27
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 28
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 29
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 30
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 31
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 32
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 33
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 34
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 35
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 36
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 37
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 38
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 39
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 40
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 41
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 42
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 43
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 44
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 45
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 46
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 47
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 48
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 49
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 50
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 51
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 52
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 53
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 54
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 55
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 56
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 57
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 58
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 59
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 60
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 61
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 62
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 63
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 64
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 65
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 66
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 67
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 68
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 69
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 70
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 71
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 72
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 73
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 74
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 75
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 76
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 77
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 78
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 79
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 80
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 81
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 82
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 83
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 84
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 85
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 86
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 87
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 88
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 89
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 90
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 91
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 92
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 93
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 94
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 95
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 96
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 97
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 98
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 99
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 100
Track 1: ['m1']
Rack 2: []
Track 3: []
Period: 0
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 1
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 2
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 3
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 4
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 5
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 6
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 7
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 8
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 9
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 10
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 11
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 12
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 13
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 14
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 15
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 16
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 17
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 18
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 19
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 20
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 21
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 22
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 23
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 24
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 25
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 26
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 27
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 28
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 29
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 30
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 31
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 32
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 33
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 34
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 35
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 36
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 37
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 38
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 39
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 40
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 41
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 42
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 43
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 44
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 45
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 46
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 47
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 48
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 49
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 50
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 51
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 52
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 53
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 54
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 55
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 56
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 57
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 58
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 59
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 60
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 61
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 62
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 63
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 64
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 65
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 66
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 67
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 68
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 69
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 70
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 71
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 72
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 73
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 74
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 75
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 76
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 77
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 78
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 79
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 80
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 81
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 82
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 83
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 84
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 85
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 86
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 87
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 88
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 89
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 90
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 91
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 92
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 93
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 94
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 95
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 96
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 97
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 98
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 99
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 100
Track 1: ['m1']
Rack 2: []
Track 3: []
Period: 0
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 1
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 2
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 3
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 4
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 5
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 6
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 7
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 8
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 9
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 10
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 11
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 12
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 13
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 14
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 15
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 16
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 17
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 18
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 19
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 20
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 21
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 22
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 23
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 24
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 25
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 26
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 27
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 28
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 29
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 30
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 31
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 32
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 33
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 34
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 35
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 36
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 37
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 38
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 39
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 40
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 41
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 42
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 43
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 44
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 45
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 46
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 47
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 48
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 49
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 50
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 51
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 52
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 53
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 54
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 55
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 56
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 57
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 58
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 59
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 60
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 61
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 62
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 63
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 64
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 65
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 66
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 67
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 68
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 69
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 70
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 71
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 72
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 73
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 74
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 75
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 76
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 77
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 78
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 79
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 80
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 81
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 82
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 83
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 84
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 85
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 86
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 87
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 88
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 89
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 90
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 91
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 92
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 93
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 94
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 95
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 96
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 97
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 98
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 99
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 100
Track 1: ['m1']
Rack 2: []
Track 3: []
Period: 0
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 1
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 2
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 3
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 4
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 5
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 6
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 7
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 8
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 9
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 10
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 11
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 12
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 13
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 14
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 15
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 16
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 17
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 18
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 19
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 20
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 21
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 22
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 23
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 24
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 25
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 26
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 27
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 28
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 29
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 30
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 31
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 32
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 33
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 34
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 35
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 36
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 37
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 38
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 39
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 40
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 41
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 42
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 43
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 44
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 45
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 46
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 47
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 48
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 49
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 50
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 51
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 52
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 53
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 54
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 55
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 56
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 57
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 58
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 59
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 60
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 61
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 62
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 63
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 64
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 65
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 66
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 67
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 68
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 69
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 70
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 71
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 72
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 73
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 74
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 75
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 76
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 77
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 78
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 79
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 80
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 81
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 82
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 83
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 84
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 85
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 86
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 87
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 88
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 89
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 90
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 91
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 92
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 93
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 94
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 95
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 96
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 97
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 98
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 99
Track 1: ['m1']
Rack 2: []
Track 3: []
=============================
Action: [3, 2, 1]
Period: 100
Track 1: ['m1']
Rack 2: []
Track 3: []
